In [75]:
%run setup_path.py
%load_ext autoreload
%autoreload 2

from os import path

from gorbslam.orbslam_processor import ModelType, ORBSLAMProcessor

# TRAJECTORY_NAME = 'seq_01'
# TRAJECTORY_NAME = 'seq_02'
TRAJECTORY_NAME = 'seq_03'

model_type = ModelType.FCNN
# model_type = ModelType.NN_CLR
# model_type = ModelType.GBR
# model_type = ModelType.SVR
# model_type = ModelType.RFR
# model_type = ModelType.UMEYAMA

processor = ORBSLAMProcessor(path.join('..', 'data','sequences', TRAJECTORY_NAME), model_type)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
# processor.initialize_model(overwrite=True)
processor.initialize_model()
processor.fit_trajectories()
processor.save_trajectories()


88/88 [==============================] - 0s 1ms/step


In [77]:
# processor.model.model.summary()


In [78]:
processor.create_ape_plot_all(include_slam=False).show()


In [79]:
# processor.create_ape_plot().show()


In [80]:
# processor.create_ape_plot(0).show()


In [81]:
# processor.create_2d_plot_utm().show()


In [82]:
# processor.create_2d_plot_slam().show()


In [83]:
# processor.create_2d_plot_slam_with_gt().show()


In [84]:
# processor.create_map_plot().show()


In [85]:
# processor.create_3d_plot_slam().show()
